In [1]:
%pip install cmake

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install dlib

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import cv2

# Load and preprocess the dataset
def load_dataset(dataset_dir):
    X = []
    y = []
    for label, category in enumerate(['closed_eyes', 'open_eyes']):
        category_dir = os.path.join(dataset_dir, category)
        for image_file in os.listdir(category_dir):
            image_path = os.path.join(category_dir, image_file)
            image = Image.open(image_path)
            image = image.resize((64, 64))  # Resize the image to a consistent size
            image = np.array(image) / 255.0  # Normalize pixel values
            X.append(image)
            y.append(label)
    return np.array(X), np.array(y)

def eye_aspect_ratio(eye):
    if len(eye) >= 6:
        A = np.linalg.norm(eye[1] - eye[5])
        B = np.linalg.norm(eye[2] - eye[4])
        C = np.linalg.norm(eye[0] - eye[3])
        ear = (A + B) / (2.0 * C)
        return ear
    else:
        return None

X, y = load_dataset(r'C:\Users\DELL\Downloads\Drowsiness Detection Dataset\train')

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert images to grayscale
X_train_gray = np.expand_dims(X_train, axis=-1)
X_val_gray = np.expand_dims(X_val, axis=-1)

# Define the CNN model for grayscale images
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN model
model.fit(X_train_gray, y_train, validation_data=(X_val_gray, y_val), epochs=10, batch_size=32)

# Save the CNN model
model.save('drowsiness_model_gray.h5')

# Train an SVM model
svm = cv2.ml.SVM_create()
svm.setKernel(cv2.ml.SVM_LINEAR)
svm.setType(cv2.ml.SVM_C_SVC)
svm.train(X_train_gray.reshape(-1, 64*64).astype(np.float32), cv2.ml.ROW_SAMPLE, y_train.astype(np.int32))

# Save the trained SVM model to a file
svm.save('eye_state_classifier_gray.xml')

# Deploy the models for real-time drowsiness detection
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    for (x, y, w, h) in eyes:
        eye = gray[y:y+h, x:x+w]
        eye_color = frame[y:y+h, x:x+w]
        eye = cv2.resize(eye, (64, 64))
        eye_color = cv2.resize(eye_color, (64, 64))
        eye = np.array(eye) / 255.0
        eye = np.expand_dims(eye, axis=-1)
        
        # Calculate eye aspect ratio (EAR)
        landmarks = eye_cascade.detectMultiScale(eye_color, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        if len(landmarks) > 0:
            ear = eye_aspect_ratio(landmarks[0])  # Use the first detected eye
            if ear is not None and ear < 0.2:  # Adjust this threshold as needed
                cv2.putText(frame, 'Drowsy', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    
            
        # Use the CNN model for prediction
        cnn_prediction = model.predict(eye.reshape(1, 64, 64, 1))
        cnn_label = np.argmax(cnn_prediction)
        
        # Use the SVM model for prediction
        _, svm_prediction = svm.predict(eye.reshape(1, -1).astype(np.float32))
        svm_label = int(svm_prediction)
        
        # Display the result
        if cnn_label == 1 and svm_label == 1:
            cv2.putText(frame, 'Open', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        else:
            cv2.putText(frame, 'Closed', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    
    cv2.imshow('Drowsiness Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Epoch 1/10
100/100 [==============================] - 15s 129ms/step - loss: 0.2369 - accuracy: 0.8894 - val_loss: 0.0602 - val_accuracy: 0.9800
Epoch 2/10
100/100 [==============================] - 16s 161ms/step - loss: 0.0534 - accuracy: 0.9812 - val_loss: 0.0672 - val_accuracy: 0.9787
Epoch 3/10
100/100 [==============================] - 17s 168ms/step - loss: 0.0333 - accuracy: 0.9903 - val_loss: 0.0290 - val_accuracy: 0.9912
Epoch 4/10
100/100 [==============================] - 16s 157ms/step - loss: 0.0185 - accuracy: 0.9947 - val_loss: 0.0224 - val_accuracy: 0.9912
Epoch 5/10
100/100 [==============================] - 16s 158ms/step - loss: 0.0162 - accuracy: 0.9947 - val_loss: 0.0091 - val_accuracy: 1.0000
Epoch 6/10
100/100 [==============================] - 16s 159ms/step - loss: 0.0081 - accuracy: 0.9984 - val_loss: 0.0022 - val_accuracy: 1.0000
Epoch 7/10
100/100 [==============================] - 16s 158ms/step - loss: 0.0043 - accuracy: 0.9991 - val_loss: 0.0015 - val_ac

c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 25ms/step
